In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import DBSCAN, KMeans
from numpy import dot
from numpy.linalg import norm

In [8]:
data_cat = pd.read_csv("data_cat.csv")
data_nom = pd.read_csv("data_nom.csv")
data_lan = pd.read_csv("data_nom.csv")

In [9]:
# removemos income por la baja cantidad de datos. no se si quedamos en eso, de u;ltoima que vuelva
data_nom.drop("income", axis = 1)

,age,height,latitude,longitude
0,22,75.0,37.653540,-122.416866
1,35,70.0,37.804456,-122.271356
2,38,68.0,37.779026,-122.419906
3,23,71.0,37.875350,-122.239634
4,29,66.0,37.779026,-122.419906
...,...,...,...,...
59941,59,62.0,37.804456,-122.271356
59942,24,72.0,37.779026,-122.419906
59943,42,71.0,37.653540,-122.416866
59944,27,73.0,37.779026,-122.419906


In [10]:
# Vamos a usar datos normalisados para los nominales
scaler = StandardScaler()
data_nom_scaled = scaler.fit_transform(data_nom)
data_nom_scaled = pd.DataFrame(data_nom_scaled, columns=data_nom.columns)
data_nom_scaled

,age,height,income,latitude,longitude
0,-1.093898,1.666782,-0.205806,-0.362416,-0.064296
1,0.281370,0.424433,0.616021,0.094853,0.001786
2,0.598740,-0.072507,-0.205806,0.017802,-0.065676
3,-0.988108,0.672903,-0.000341,0.309659,0.016193
4,-0.353369,-0.569446,-0.205806,0.017802,-0.065676
...,...,...,...,...,...
59941,2.820327,-1.563325,-0.205806,0.094853,0.001786
59942,-0.882318,0.921372,-0.205806,0.017802,-0.065676
59943,1.021900,0.672903,0.821475,-0.362416,-0.064296
59944,-0.564949,1.169842,-0.205806,0.017802,-0.065676


In [11]:
# La idea es no incluir sexo y orientacion en el calculo de distancia: los guardamos separado para no incidir en las agrupaciones
# pregunta y "estado"? si lo dejamos en distancia va a juntar solteros con solteros y casados con casados verque onda

data_sex = data_cat.pop("sex")
data_ori = data_cat.pop("orientation")

In [12]:
onehot = OneHotEncoder(sparse=False)
onehotfit = onehot.fit(data_cat)
data_cat_dummies = onehot.transform(data_cat)
data_cat_dummies = pd.DataFrame(data_cat_dummies, columns=onehot.get_feature_names(data_cat.columns))
data_cat_dummies

,status_available,status_married,status_seeing someone,status_single,status_unknown,body_type_a little extra,body_type_athletic,body_type_average,body_type_curvy,body_type_fit,...,sign_matters_not say,sign_matters_rather not say,dogs_dislikes dogs,dogs_has dogs,dogs_likes dogs,dogs_no_opinion dogs,cats_ dislikes cats,cats_ has cats,cats_ likes cats,cats_ no_opinion cats
0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59941,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
59942,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
59943,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
59944,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [13]:
#uno los dats en un solo vector por persona:

data_all = pd.merge(data_nom_scaled, data_cat_dummies,how="inner", left_index=True, right_index=True)
data_all = pd.merge(data_all,data_lan,how="inner", left_index=True, right_index=True)

In [15]:
# defino una function de distancia entre personas: 

def distanciaEntrePersonas(persona1, persona2):
    # segun google esto deberia andar. no se como chequearlo. por lo pronto corre
    return np.dot(persona1, persona2) / (np.linalg.norm(persona1) * np.linalg.norm(persona2))

In [19]:
#busco los mejores N candidatos:

numero_recomendaciones = 10 #para tirar un corte y que no me recomiende a todes :P 

recomendaciones_x_cliente = []

for cliente in data_all:

    scores = [] # lista de scores

    for idx, persona in data_all.iterrows():

        dist = distanciaEntrePersonas(cliente, persona)
        scores.append((idx, dist))  # agrego la tupla idx, dist a una lista

    #ahora aca tengo que ordenar la lista scores en orden ascendente de distancia:

    scores.sort(key = lambda x: x[1]) #estoy ordenanjdo por el segundo elem,ento de la tupla q es la dist again, segun google y le creo

    print (scores)

    idx_ofertas = []
    # recorrer scores y devolver las recomendaciones que complan con criterios de orientacion y sexo

    for idx, score in scores:

        if cliente.sex #noinono -.- :
            idx_ofertas.append(idx)

        if len(idx_ofertas) >= numero_recomendaciones:
            break;

    # en idx_ofertas estan los indices de los mejores
    recomendaciones_x_cliente.append(idx_ofertas)
    

SyntaxError: invalid syntax (91144205.py, line 27)